### Deep learning inference

- Detect tree objects from dataset

In [ ]:
from urbantree.setting import Setting

#SETTING = "settings/by_dop80c_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/by_dop80c_1312/deepforest_r2/setting.yaml"
SETTING = "settings/by_dop80c_1312/deepforest_r3/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r2/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r3/setting.yaml"

setting = Setting.load_deepforest_setting(SETTING)
setting['model_inference_config']

- detect tree objects from dataset and the resulting bounding boxes DataFrame are saved in pickle objects.
- The object detection can be performed on different patch sizes and
  no extra nms is run on aggregated results.

In [ ]:
from urbantree.deepforest.detection import infer_images

infer_images(**setting)

- Render tree canopy raster images according to the resulting bounding boxes DataFrame saved in pickle objects.

In [ ]:
from urbantree.deepforest.detection import postprocess_render_images

dataset_img_pattern="*.tiff"
#dataset_img_pattern = "*6128569*"
#dataset_img_pattern = "*1287912.570476842_*"

postprocess_render_images(**setting,
                          dataset_img_pattern=dataset_img_pattern)

- calculate difference among two datasets

In [1]:
from urbantree.deepforest.detection import calc_diff

calc_diff(diff_from_setting_path="settings/opendata_luftbild_dop60_1312/deepforest_r2/setting.yaml",
          diff_to_setting_path="settings/by_dop80c_1312/deepforest_r3/setting.yaml",
          diff_from_score_factor=1.8,
          diff_to_score_factor=1.0,
          diff_from_size_threshold=140,
          aggregate_iou_threshold=0.25,
          diff_iou_threshold=0.1,
          output_bbox_dir='temp/diff3/b',
          output_img_dir='temp/diff3/debug') 

In [ ]:
from urbantree.deepforest.detection import create_bbox_geojson

create_bbox_geojson(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_dif='temp/diff3/b',
                    output_geojson_path='temp/diff3/diff.geojson',
                    iou_threshold=0.3)